In [1]:
from qiskit_algorithms import MinimumEigensolverResult
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit.circuit.library import EfficientSU2
import numpy as np
from qiskit_algorithms.optimizers import SLSQP , SPSA , ADAM
from qiskit_aer.primitives import Estimator
#qiskit_nature.settings.use_pauli_sum_op = False 
from qiskit_aer import Aer
from pyinstrument import Profiler
import threading
from qiskit_aer.primitives import Estimator as AerEstimator
import math
from qiskit.circuit.library import EfficientSU2
from qiskit_aer import AerSimulator
import cupy as cp

In [4]:
molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Li", "H"],
        coords=([0.0, 0.0, 0.0], [3.6895640, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

In [5]:
driver = PySCFDriver.from_molecule(molecule)
properties = driver.run()

problem = FreezeCoreTransformer(
    freeze_core=True, remove_orbitals=[-3, -2]
).transform(properties)

num_particles = problem.num_particles
num_spatial_orbitals = problem.num_spatial_orbitals

mapper = ParityMapper(num_particles=num_particles)
qubit_op = mapper.map(problem.second_q_ops()[0])

print(qubit_op)



SparsePauliOp(['IIIIIIII', 'IIIIIIIZ', 'IIIIIIZZ', 'IIIIIIZI', 'IIIIXXXZ', 'IIIIXXXI', 'IIIIYXYI', 'IIIIYXYZ', 'IIIIIZZI', 'IIIIIZZZ', 'IIIIZZII', 'IIIIZZIZ', 'IIIIZIII', 'IIIIZIIZ', 'IIIZIIII', 'IIIZIIIZ', 'IIZXIIII', 'IIZXIIIZ', 'IIIXIIII', 'IIIXIIIZ', 'XXXXIIII', 'XXXXIIIZ', 'YXXYIIII', 'YXXYIIIZ', 'IIZZIIII', 'IIZZIIIZ', 'XXXZIIII', 'XXXZIIIZ', 'YXYIIIII', 'YXYIIIIZ', 'IZZIIIII', 'IZZIIIIZ', 'ZZIIIIII', 'ZZIIIIIZ', 'ZIIIIIII', 'ZIIIIIIZ', 'IIIIXXXX', 'IIIIXXYY', 'IIIIYXXY', 'IIIIYXYX', 'IIIIIIZX', 'IIIIIZIX', 'IIIIIIIX', 'IIIIIZZX', 'IIIIZZZX', 'IIIIZZIX', 'IIIIZIZX', 'IIIIZIIX', 'IIIZIIZX', 'IIIZIIIX', 'IIZXIIZX', 'IIIXIIZX', 'IIZXIIIX', 'IIIXIIIX', 'XXXXIIZX', 'YXXYIIZX', 'XXXXIIIX', 'YXXYIIIX', 'IIZZIIZX', 'IIZZIIIX', 'XXXZIIZX', 'YXYIIIZX', 'XXXZIIIX', 'YXYIIIIX', 'IZZIIIZX', 'IZZIIIIX', 'ZZIIIIZX', 'ZZIIIIIX', 'ZIIIIIZX', 'ZIIIIIIX', 'IIIIXYYX', 'IIIIYYYY', 'IZXXIZXX', 'IIYYIZXX', 'IZXXIIYY', 'IIYYIIYY', 'IZXZIZXX', 'IIXIIZXX', 'IZXZIIYY', 'IIXIIIYY', 'XXZIIZXX', 'YYIIIZXX', '

In [6]:
init_state = HartreeFock( num_spatial_orbitals, num_particles, mapper)
var_form = UCCSD( num_spatial_orbitals, num_particles, mapper, initial_state=init_state)
# var_form = EfficientSU2(num_qubits=qubit_op.num_qubits, entanglement='linear' , initial_state=init_state)
print(var_form)

     »
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
q_4: »
     »
q_5: »
     »
q_6: »
     »
q_7: »
     »
«     ┌────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                   ├
«     │                                                                                                                                                    │
«q_1: ┤1                                                                                                                                                   ├
«     │                                                                                                                                                    │
«q_2: ┤2                                                                                        

In [7]:

optimizer = SLSQP(maxiter=30)
aer_estimator = Estimator(approximation=True)


In [8]:
def exact_result(qubit_op, problem):

    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

result = exact_result(qubit_op, problem)
print(" exact energy " , result.total_energies[0].real)

 exact energy  -7.785815867331454


In [9]:
def exact_result(qubit_op, problem):
    
    matrix = qubit_op.to_matrix()
    cp_matrix = cp.array(matrix)

    eigenvalues, eigenvectors = cp.linalg.eigh(cp_matrix)
    min_index = cp.argmin(eigenvalues.real)
    min_eigenvalue = eigenvalues[min_index].real
    min_eigenvector = eigenvectors[:, min_index]

    min_eigenvalue = cp.asnumpy(min_eigenvalue)
    min_eigenvector = cp.asnumpy(min_eigenvector)

    result = MinimumEigensolverResult()
    result.eigenvalue = min_eigenvalue
    result.eigenstate = min_eigenvector

    interpreted_result = problem.interpret(result)
    return interpreted_result



result = exact_result(qubit_op, problem)
print(" exact energy " , result.total_energies[0].real)

 exact energy  -7.785815867331451


In [10]:
vqe = VQE(aer_estimator, var_form, optimizer)
vqe.initial_point = [0] * var_form.num_parameters
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

algorithm = GroundStateEigensolver(mapper, vqe)

electronic_structure_result = algorithm.solve(problem)
electronic_structure_result.formatting_precision = 6
print(electronic_structure_result.groundenergy)
print(electronic_structure_result)

-0.7938994953962971
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -8.216092
  - computed part:      -0.793899
  - FreezeCoreTransformer extracted energy part: -7.422192
~ Nuclear repulsion energy (Hartree): 0.430276
> Total ground state energy (Hartree): -7.785816
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: -0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [6.972265  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [7.171604  0.000001  -0.000001]
    - computed part:      [7.174043  0.000001  -0.000001]
    - FreezeCoreTransformer extracted energy part: [-0.002439  0.0  0.0]
  > Dipole moment (a.u.): [-0.199338  -0.000001  0.000001]  Total: 0.199338
                 (debye): [-0.506668  -0.000002  0.000002]  Total: 0.506668
 


In [11]:
vqe =  VQE(aer_estimator , var_form , optimizer,initial_point=[0] * var_form.num_parameters)

vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
vqe_result = problem.interpret(vqe_calc).total_energies[0].real
print(vqe_result)

-7.785815720595295
